In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm_notebook, tqdm
tqdm.pandas()
# from pandarallel import pandarallel
# pandarallel.initialize()

In [2]:
results = pd.read_csv('results.csv')
results.loc[:, 'mask'] = results.loc[:, 'mask'].str.replace('X', '')
roster = pd.read_csv('roster.csv').set_index('idplayer')
tournaments = pd.read_csv('tournaments.csv').set_index('idtournament')
tournaments = tournaments.loc[~tournaments.questions_total.isnull(), :]
common_tours = list(set(tournaments.index).intersection(set(results.idtournament)))
results = results.loc[results.idtournament.isin(common_tours), :]
tournaments = tournaments.loc[common_tours, :]

q_total = pd.DataFrame(results.groupby('idtournament').first()\
                       .loc[:, 'mask'].str.len()).dropna().rename(columns={'mask': 'questions_total'})
tournaments = pd.merge(tournaments.drop(columns=['questions_total']), q_total, left_index=True, right_index=True)

In [3]:
ineligible_tournaments = [1521, 1526, 1531, 1534, 1780, 1817, 1849, 1854, 1858, 1885, 1893, 1902, 1925, 1926, 1928,
                          1931, 1960, 2025, 2040, 2077, 2087, 2128, 2147, 2151, 2166, 2183, 2291, 2345, 2346, 2472,
                          2479, 2512, 2521, 2578, 2611, 2707, 2732, 2763, 2783, 2833, 3130, 3162, 3231, 3510, 3589,
                          3590, 3617, 4011, 4169, 4375, 4469, 4481, 4497, 4713, 4714, 4816, 4860, 4881, 4913, 4946,
                          4969, 4983, 4984, 4989, 4990, 4992, 5032, 5054, 5097, 5102, 5106, 5121, 5126, 5160, 5178,
                          5210, 5215, 5221, 5239, 5241, 5247, 5299, 5310]
results = results.loc[~results.idtournament.isin(ineligible_tournaments), :]
results.loc[:, 'mask'] = results.loc[:, 'mask'].apply(lambda x: list(str(x)))
roster = roster.loc[~roster.idtournament.isin(ineligible_tournaments), :]

In [4]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    orders = [num for range_len in lens for num in range(range_len)]
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    
    res.loc[:, 'q_numb'] = orders
    return res

In [5]:
exp = explode(results.loc[:, ['idteam', 'idtournament', 'mask']], ['mask'], fill_value='', preserve_index=False)\
            .rename(columns={'mask': 'q_taken'})
exp.head(3)

,idteam,idtournament,mask,q_numb
0,209,3236,1,0
1,209,3236,1,1
2,209,3236,1,2
